# Introduction à l'optimisation différentiable : méthodes de descente de gradients

Le but du TP est d'illustrer les performances d'un certain nombre de méthodes classiques pour l'optimisation sans contrainte, dans le cas de problèmes quadratiques et non-linéaires. 

In [ ]:
#
# Chargement des librairies
#
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as la
from scipy import linalg as sla
import math
import functools as ft

 ## Définition des problèmes d'optimisation

Les problèmes étudiés seront les suivants :
 
   $(\mathcal{P}_1)\quad \min_{x \in \mathbb{R}^n} f(x)=x^TAx+b^Tx$, avec $A$ symétrique définie positive;
   
   $(\mathcal{P}_2)\quad \min_{x \in \mathbb{R}^2} f(x)=100(x_2-x_1^2)^2+(1-x_1)^2$, avec f la fonction de Rosenbrock.
  
La minimisation de fonctionnelles quadratiques (problème $(\mathcal{P}_1)$) constitue la base sur lequel s'appuie un certain nombre de méthodes d'optimisation. Le problème $(\mathcal{P}_2)$ permet d'illustrer les difficultés qui surgissent quand on cherche à minimsier une fonction non-linéaire non-quadratique.

## I - Minimisation d'une fonction quadratique

In [ ]:
## Problèmes quadratiques
def build_problem(n,condA):
    # Construction d'une matrice symétrique définie positive
    # et d'un vecteur de même dimension
    # n : entier, dimension de la matrice
    # condA : entier, conditionnement de la matrice 10^condA
    
    A = np.zeros([n,n])
    Q,R=sla.qr(np.random.rand(n,n))
    D=np.diag((10*np.ones([n]))**np.linspace(-condA,0,num=n))
    A=np.matmul(np.matmul(np.transpose(Q),D),Q)    
    
    b=np.random.randn(n,1)
    
    return A,b

# Paramètres du problème
condA=3
n=100
A,b=build_problem(n,condA)


**Question**: Implanter les fonctions f, gradient de f et Hessienne de f.

In [ ]:
def fquad(x,A,b):
    # Fonction quadratique : f(x)=0.5*x'*A*x+b'*x =x'*(0.5*(A*x+b))
    # A : matrice symétrique de taille n
    # b,x : vecteurs de taille n
    f=np.dot(np.transpose(x),(0.5*np.dot(A,x)+b))
    return f

def gquad(x,A,b):
    # Gradient de la fonction fquad
    # A : matrice symétrique de taille n
    # b,x : vecteurs de taille n
    
    g=np.dot(A,x)+b
    return g

def hquad(x,A,b):
    # Gradient de la fonction fquad
    # A : matrice symétrique de taille n
    # b,x : vecteurs de taille n
    return A

 ### I-1 Application de la méthode de la plus grande pente ("steepest descent").
 
 **Question**: Implanter la méthode de la steepest descent pour la minimisation d'une fonction quadratique.

In [ ]:
def pas_steep(x,dd,h):
    # Calcul le pas de la steepest descent  
    d=dd[:,0]
    num=np.dot(np.transpose(d),d)
    den=np.dot(np.transpose(d),np.dot(h(x),d))
    alpha=num/den
    return alpha

def descente(x0,param,f,g,pas,fpas):
    # Fonction steepest descent
    # Input : 
    # x0 : point de départ
    # param=[tol1,tol2,itermax]
    # A, b : données du problème
    # Output :
    # x : optimum
    # flag : 1 - convergence, 2 - stagnation, 3 - itermax
    # nbiter : nombre d'itération
    
    tolg=param[0]
    tols=param[1]
    itermax=param[2]
    
    
    x=x0
    d=-g(x0)
    ngrad0=la.norm(d)    
    ng=ngrad0
    
    iterf=np.array([f(x)])
    flag=0
    nbiter=0
    
    n=len(x0)
    dd=np.zeros([n,2])
    
    while flag == 0:  
        nbiter=nbiter+1
        #calcul du pas
        #den=np.dot(np.transpose(d),np.dot(A,d))
        #alpha=ng**2/den
        
        dd[:,0]=d[:,0]
        dd[:,1]=-d[:,0]
        alpha=pas(x,dd,fpas)
        
        #mise à jour
        nxold=la.norm(x)
        x=x+alpha*d 
        iterf=np.concatenate((iterf,np.array([f(x)])))
        
        # calcul de la nouvelle direction
        d=-g(x)
        ng=la.norm(d)
    
        if ng <= tolg*(ngrad0+10**(-14)):
            flag=1
        elif alpha*ng <= tols*(nxold+10**(-14)): 
            flag=2
        elif nbiter == itermax:
            flag=3
    # Fin boucle
      
    return x,flag,nbiter,iterf


**Question**: Résoudre le problème d'optimisation $(\mathcal{P}_1)$. Evaluez la sensibilité des résultats aux paramètres d'entrée de l'algorithme ($x_0$, tolérances, nombre d'itérations maximum,..).

In [ ]:
x0=np.random.randn(n,1)#np.ones([n,1])
param=[10**(-6),10**(-12),10000]

floc=ft.partial(fquad,A=A,b=b)
gloc=ft.partial(gquad,A=A,b=b)
hloc=ft.partial(hquad,A=A,b=b)

x,flag,nbiter,iterf=descente(x0,param,floc,gloc,pas_steep,hloc)

tmp=sla.solve(A,-b)
fopt=0.5*np.dot(np.transpose(b),tmp)
print("Valeur optimale théorique de f:",fopt)
print("Valeur à l'optinum numérique:", floc(x))
print("Condition d'arrêt:", flag)
if flag > 1:
    print("Norme du gradient initial:",la.norm(gloc(x0)))
    print("Norme du gradient:",la.norm(gloc(x)))

plt.plot(iterf[:,0,0])
plt.ylabel('f')
plt.xlabel('Itérations')
plt.show()

**Question**: Illuster l'impact du conditionnement de $A$ sur les performances de l'algorithme.

In [ ]:
x0=np.random.randn(n,1)
param=[10**(-6),10**(-12),250000]
iterC=np.array([])
condAmax=6

for condA in range(condAmax):
    A,b0=build_problem(n,condA)
    
    floc=ft.partial(fquad,A=A,b=b)
    gloc=ft.partial(gquad,A=A,b=b)
    hloc=ft.partial(hquad,A=A,b=b)
        
    x,flag,nbiter,iterf=descente(x0,param,floc,gloc,pas_steep,hloc)
    
    iterC=np.concatenate((iterC,np.array([nbiter])))

xx=np.log10((10*np.ones([condAmax]))**np.linspace(1,condAmax,num=condAmax))
plt.plot(xx,iterC)
plt.ylabel('Nb itérations')
plt.xlabel('Conditionnement')
plt.show()

 ### I-2 Application de la méthode de Newton.
 
 **Question**: Implanter la méthode de Newton pour la minimisation d'une fonction quadratique. Vous utiliserez la méthode du gradient conjugué pour résoudre le système linéaire associé. 

In [ ]:
def CG(x0,A,b,tol,itermax):
    #Gradient conjugé
    
    x=x0
    niter=0
    r=b-np.dot(A,x)
    d=r
    nb=la.norm(b)
    delta_new=np.dot(np.transpose(r),r)
    iterf=np.array([math.sqrt(delta_new)])
    convergence=0
    
    while convergence ==0:
        niter=niter+1
       
        #pas optimal
        q=np.dot(A,d)
        den=np.dot(np.transpose(d),q)
        alpha=delta_new/den
        
        #mise à jour
        x=x+alpha*d
        r=r-alpha*q
    
        delta_old=delta_new
        delta_new=np.dot(np.transpose(r),r)
       
        #nouvelle direction
        beta=delta_new/delta_old
        d=r+beta*d
        
        #Test d'arrêt
        if math.sqrt(delta_new) <= tol*(nb+10**(-14)):
            convergence=1
        elif niter == itermax:
            convergence=2
        
        iterf=np.concatenate((iterf,np.array([math.sqrt(delta_new)])))
        
    #fin boucle
    return x,iterf,niter,convergence    

In [ ]:
def Newton(x0,param,f,g,h):
    # Fonction résoltuion par méthode de Newton
    # Input : 
    # x0 : point de départ
    # param=[tol1,tol2,itermax]
    # A, b : données du problème
    # Output :
    # x : optimum
    # flag : 1 - convergence, 2 - stagnation, 3 - itermax
    # nbiter : nombre d'itération
    
    tolg=param[0]
    tols=param[1]
    itermax=param[2]
        
    x=x0
    n=len(x0)
    ngrad0=la.norm(g(x))        
    iterf=np.array([f(x)])
    iterCG=0
    
    flag=0
    nbiter=0
    
    while flag == 0:  
        nbiter=nbiter+1
        # calcul de la nouvelle direction
        gcur=g(x)
        ng=la.norm(gcur)
       
        #d,info=spla.cg(h(x,A,b),-gcur,np.zeros([n,1]),10**(-6),10**4)
        d,itercg,niterCG,flagCG=CG(np.zeros([n,1]),h(x),-gcur,10**(-6),10**4)      
        iterCG=iterCG+niterCG
        
        #mise à jour
        nxold=la.norm(x)
        # x[:,0]=x[:,0]+d
        x=x+d
        
        iterf=np.concatenate((iterf,np.array([f(x)])))
        
        #tests arret
        if ng <= tolg*(ngrad0+10**(-14)):
            flag=1
        elif la.norm(d) <= tols*(nxold+10**(-14)): 
            flag=2
        elif nbiter == itermax:
            flag=3
    # Fin boucle
      
    return x,flag,nbiter,iterf,iterCG


In [ ]:
# Paramètres du problème
condA=3
n=1000
A,b=build_problem(n,condA)

floc=ft.partial(fquad,A=A,b=b)
gloc=ft.partial(gquad,A=A,b=b)
hloc=ft.partial(hquad,A=A,b=b)

x0=np.random.randn(n,1)#np.ones([n,1])
param=[10**(-6),10**(-12),10000]
x,flag,nbiter,iterf,iterCG=Newton(x0,param,floc,gloc,hloc)

tmp=sla.solve(A,-b)
fopt=0.5*np.dot(np.transpose(b),tmp)
print("Valeur optimale théorique de f:",fopt)
print("Valeur à l'optinum numérique:", floc(x))
print("Condition d'arrêt:", flag)
if flag > 1:
    print("Norme du gradient initial:",la.norm(gloc(x)))
    print("Norme du gradient:",la.norm(gloc(x)))

plt.plot(iterf[:,0,0])
plt.ylabel('f')
plt.xlabel('Itérations')
plt.show()

**Question**: Illustrer l'impact du conditionnement de $A$ sur les performances de l'algorithme. Vous afficherez le nombre d'itérations de la méthode de Newton et le nombre d'itérations du gradient conjugué.

In [ ]:
x0=np.random.randn(n,1)
param=[10**(-6),10**(-12),100000]
iterC=np.array([])
iterTCG=np.array([])
condAmax=8

for condA in range(condAmax):
    A,b0=build_problem(n,condA)
    
    floc=ft.partial(fquad,A=A,b=b)
    gloc=ft.partial(gquad,A=A,b=b)
    hloc=ft.partial(hquad,A=A,b=b)
    
    x,flag,nbiter,iterf,iterCG=Newton(x0,param,floc,gloc,hloc)
    iterC=np.concatenate((iterC,np.array([nbiter])))
    iterTCG=np.concatenate((iterTCG,np.array([iterCG])))
    
xx=np.log10((10*np.ones([condAmax]))**np.linspace(1,condAmax,num=condAmax))

plt.subplot(131)
plt.plot(xx,iterC)
plt.ylabel('Nb itérations Newton')
plt.xlabel('Conditionnement')

plt.subplot(133)
plt.plot(xx,iterTCG)
plt.ylabel('Nb itérations CG')
plt.xlabel('Conditionnement')

plt.show()

## II - Minimisation d'une fonction non-linéaire

In [ ]:
## Problèmes non-linéaires : fonction de Rosenbrock
def f2(x):
    # fonction de Rosenbrock de R^2 dans R
    # x vecteur de taille 2
    f=100*(x[1]-x[0]**2)**2+(1-x[0])**2
    return f

def g2(x):
    # gradient de la fonction de Rosenbrock
    # x vecteur de taille 2
    g=np.zeros([2,1])
    g[0]=-400.0*x[0]*(x[1]-x[0]**2)-2*(1-x[0])
    g[1]=200.0*(x[1]-x[0]**2)
    return g

def h2(x):
    # Hessienne de la fonction de Rosenbrock
    # x vecteur de taille 2
    H=np.zeros([2,2])
    H[0,0]=1200*x[0]**2+2-400*x[1]
    H[0,1]=-400*x[0]
    H[1,0]=H[0,1]
    H[1,1]=200.
    return H

**Question**: Implanter des stratégies de pas et appliquer la méthode de descente de gradient à la fonction de Rosenbrock.

In [ ]:
def pas_cst(x,dd,f):
    # Pas constant
    alpha=0.01
    return alpha

###################
def flaff(x,a,b):
    return a*x+b

def pas_bckt(x,dd,f):
    # Backtracking
    # dd contient la direction d et le gradient en x: dd[:,0]=d, dd[:,1]=g(x)
    rho=0.5
    alpha=1
    c1=0.1
    
    n=len(x)
    d=np.zeros([n,1])
    g=np.zeros([n,1])
    d[:,0]=dd[:,0]
    g[:,0]=dd[:,1]
    
    a=c1*np.dot(np.transpose(g),d)
    b=f(x)
    
    while f(x+alpha*d) > flaff(alpha,a,b):
        alpha=rho*alpha
    
    #print("bckt:",alpha)
    return alpha
    

In [ ]:
# Paramètres du problème
n=2
#x0=np.random.randn(n,1)#np.ones([n,1])
xopt=np.ones([n,1])
sigma_b=0.1
x0=xopt+sigma_b*np.random.randn(n,1)
param=[10**(-6),10**(-12),10000]

choix_pas=2
if choix_pas == 1:
    #pas constant
    x,flag,nbiter,iterf=descente(x0,param,f2,g2,pas_cst,f2)
elif choix_pas == 2:
    #bakctracking
    x,flag,nbiter,iterf=descente(x0,param,f2,g2,pas_bckt,f2)

print("Valeur optimale théorique de f:",f2([1,1]))
print("Valeur à l'optinum numérique:", f2(x))
print("Condition d'arrêt:", flag)
if flag > 1:
    print("Norme du gradient initial:",la.norm(g2(x0)))
    print("Norme du gradient:",la.norm(g2(x)))

    
plt.plot(iterf[:,0])
plt.ylabel('f')
plt.xlabel('Itérations')
plt.show()

**Question**: Modifier le code l'algorithme Newton pour inclure la recherche de pas. Evaluer les performances de cette approche.

In [ ]:
def Newton_pas(x0,param,f,g,h,pas,fpas):
    # Fonction résoltuion par méthode de Newton
    # Input : 
    # x0 : point de départ
    # param=[tol1,tol2,itermax]
    # A, b : données du problème
    # Output :
    # x : optimum
    # flag : 1 - convergence, 2 - stagnation, 3 - itermax
    # nbiter : nombre d'itération
    
    tolg=param[0]
    tols=param[1]
    itermax=param[2]
        
    x=x0
    ngrad0=la.norm(g(x))        
    iterf=np.array([f(x)])
    iterCG=0
    
    flag=0
    nbiter=0
    
    n=len(x)
    dd=np.zeros([n,2])
    
    while flag == 0:  
        nbiter=nbiter+1
        # calcul de la nouvelle direction
        gcur=g(x)
        ng=la.norm(gcur)
       
        #d,info=spla.cg(h(x,A,b),-gcur,np.zeros([n,1]),10**(-6),10**4)
        d,itercg,niterCG,flagCG=CG(np.zeros([n,1]),h(x),-gcur,10**(-6),10**4)      
        iterCG=iterCG+niterCG
        
        #calcul du pas
        dd[:,0]=d[:,0]
        dd[:,1]=-d[:,0]
        alpha=pas(x,dd,fpas)
        
        #mise à jour
        nxold=la.norm(x)
        # x[:,0]=x[:,0]+d
        x=x+alpha*d
        
        iterf=np.concatenate((iterf,np.array([f(x)])))
        
        #tests arret
        if ng <= tolg*(ngrad0+10**(-14)):
            flag=1
        elif alpha*la.norm(d) <= tols*(nxold+10**(-14)): 
            flag=2
        elif nbiter == itermax:
            flag=3
    # Fin boucle
      
    return x,flag,nbiter,iterf,iterCG


In [ ]:
# Paramètres du problème
n=2
#x0=np.random.randn(n,1)#np.ones([n,1])
xopt=np.ones([n,1])
sigma_b=1
x0=xopt+sigma_b*np.random.randn(n,1)
param=[10**(-6),10**(-12),10000]

choix_pas=2
if choix_pas == 1:
    #pas constant
    x,flag,nbiter,iterf,iterCG=Newton_pas(x0,param,f2,g2,h2,pas_cst,f2)
elif choix_pas == 2:
    #bakctracking
    x,flag,nbiter,iterf,iterCG=Newton_pas(x0,param,f2,g2,h2,pas_bckt,f2)

print("Valeur optimale théorique de f:",f2([1,1]))
print("Valeur à l'optinum numérique:", f2(x))
print("Condition d'arrêt:", flag)
if flag > 1:
    print("Norme du gradient initial:",la.norm(g2(x0)))
    print("Norme du gradient:",la.norm(g2(x)))

    
plt.plot(iterf[:,0])
plt.ylabel('f')
plt.xlabel('Itérations')
plt.show()